# 1. Which movie we can suggest to Anne ?

Build a dictionary critiques containing the movie critics and their ratings.

In [283]:
import pandas as pd
import numpy as np
import math
from math import sqrt
import random

critiques = {}

critiques['Lisa Rose'] = {'Lady': 2.5, 'Snake': 3.5, 'Luck': 3.0, 'Superman': 3.5, 'Dupree': 2.5, 'Night': 3.0}
critiques['Gene Seymour'] = {'Lady': 3.0, 'Snake':3.5, 'Luck': 1.5, 'Superman': 5.0, 'Dupree': 3.5, 'Night': 3.0}
critiques['Michael Phillips'] = {'Lady': 2.5, 'Snake':3.0, 'Luck': np.NaN, 'Superman': 3.5, 'Dupree': np.NaN, 'Night': 4.0}
critiques['Claudia Puig'] = {'Lady': np.NaN, 'Snake':3.5, 'Luck': 3.0, 'Superman': 4.0, 'Dupree': 2.5, 'Night': 4.5}
critiques['Mick Lasalle'] = {'Lady': 3.0, 'Snake':4.0, 'Luck': 2.0, 'Superman': 3.0, 'Dupree': 2.0, 'Night': 3.0}
critiques['Jack Matthews'] = {'Lady': 3.0, 'Snake':4.0, 'Luck': np.NaN, 'Superman': 5.0, 'Dupree': 3.5, 'Night': 3.0}
critiques['Toby'] = {'Lady': np.NaN, 'Snake': 4.5, 'Luck': np.NaN, 'Superman': 4.0, 'Dupree': 1.0, 'Night': np.NaN}
critiques['Anne'] = {'Lady': 1.5, 'Snake':np.NaN, 'Luck': 4.0, 'Superman': np.NaN, 'Dupree': 2.0, 'Night': np.NaN}

critiques1 = pd.DataFrame(critiques)
movies = (critiques1.index)

critiques1

,Lisa Rose,Gene Seymour,Michael Phillips,Claudia Puig,Mick Lasalle,Jack Matthews,Toby,Anne
Lady,2.5,3.0,2.5,NaN,3.0,3.0,NaN,1.5
Snake,3.5,3.5,3.0,3.5,4.0,4.0,4.5,NaN
Luck,3.0,1.5,NaN,3.0,2.0,NaN,NaN,4.0
Superman,3.5,5.0,3.5,4.0,3.0,5.0,4.0,NaN
Dupree,2.5,3.5,NaN,2.5,2.0,3.5,1.0,2.0
Night,3.0,3.0,4.0,4.5,3.0,3.0,NaN,NaN


After collecting data about the things people like, we need a way to determine how similar people are in their tastes.

(a) To compute a simple similarity score, we use the Manhattan distance or an Euclidean distance.

In [284]:
def sim_distanceManhattan(person1, person2):
    distance = 0
    for x1, x2 in zip(person1, person2):
        if np.isnan(x1) or np.isnan(x2):
            pass
        else:
            difference = x2 - x1
            absolute_difference = abs(difference)
            distance += absolute_difference

    return distance
            
sim_distanceManhattan(critiques1['Lisa Rose'], critiques1['Gene Seymour'])

4.5

In [285]:
def sim_distanceEuclidienne(person1, person2):
    
    difference = 0
    sum_of_squares = 0
    for x1, x2 in zip(person1, person2):
        if np.isnan(x1) or np.isnan(x2):
            pass
        else:
            difference = (x2 - x1) ** 2
            sum_of_squares += difference

    return sum_of_squares ** 0.5
            
sim_distanceEuclidienne(critiques1['Lisa Rose'], critiques1['Gene Seymour'])

2.3979157616563596

Build a function computeNearestNeighbor(nouveauCritique, critiques), returning a sorted list of critics close to nouveauCritique.

In [286]:
def computeNearestNeighbor(nouveauCritique, critiques1):
    
    distances=[]
    
    for critique in critiques1:
        if critique != nouveauCritique:
            distance = sim_distanceManhattan(critiques1[critique], critiques1[nouveauCritique])
            distances.append((distance, critique))
    
    distances.sort()
    return distances

computeNearestNeighbor('Anne', critiques1)

[(1.0, 'Michael Phillips'),
 (1.0, 'Toby'),
 (1.5, 'Claudia Puig'),
 (2.5, 'Lisa Rose'),
 (3.0, 'Jack Matthews'),
 (3.5, 'Mick Lasalle'),
 (5.5, 'Gene Seymour')]

Build a function recommendNearestNeighbor returning one nearest critic to nouveauCritique.

In [287]:
def recommendNearestNeighbor(nouveauCritique, Critiques, array):
    
    recommend_movies = []
    NearestNeighbor = computeNearestNeighbor(nouveauCritique, Critiques)[0][1]
    print("Nearest neighbor is {}.".format(NearestNeighbor))
    
    for i in range(0, len(Critiques[nouveauCritique])):
        if np.isnan(Critiques[nouveauCritique][i]) and not np.isnan(Critiques[NearestNeighbor][i]):
            recommend_movies.append((array[i], Critiques[NearestNeighbor][i]))
    return recommend_movies
    
recommendNearestNeighbor('Anne', critiques1, movies)

Nearest neighbor is Michael Phillips.


[('Snake', 3.0), ('Superman', 3.5), ('Night', 4.0)]

Build a function returning a list of movies that given nouveauCritique didn't see.

In [288]:
def not_seen_by_Anne(nouveauCritique, Critiques, array):
    
    not_seen_array = []
    for i in range(0, len(Critiques[nouveauCritique])):
        if math.isnan(Critiques[nouveauCritique][i]):
            not_seen_array.append(array[i])
            
    return (not_seen_array)

not_seen_by_Anne('Anne', critiques1, movies)

['Snake', 'Superman', 'Night']

Build a function calculating global score by given movie name.

In [289]:
def weighted_score(nouveauCritique, movieName, critiques):
    
    #print(critiques1.iloc[[critiques1.index.get_loc(movieName)]]) #getting row by given movie name
    
    total_a_m = 0
    s_a_m = 0
    global_score_a_m = 0
    numerator_m = 0
    d_m = 0
    
    #for i in critiques1.columns.values:
        #print(critiques1[i][critiques1.index.get_loc(movieName)]) #getting value of each critique to given movie name
    
    for critique_index in range(len(critiques.columns.values)):
        numerator_m = critiques[critiques.columns.values[critique_index]][critiques.index.get_loc(movieName)]
        if math.isnan(numerator_m):
            break
        d_m = sim_distanceManhattan(critiques[nouveauCritique],critiques[critiques.columns.values[critique_index]])
        total_a_m += numerator_m / (1 + d_m)
        s_a_m += 1 / (1 + d_m)
        global_score_a_m = total_a_m / s_a_m

    total_a_e = 0
    s_a_e = 0
    global_score_a_e = 0
    numerator_e = 0
    d_e = 0
    
    for critique_index in range(len(critiques.columns.values)):
        numerator_e = critiques[critiques.columns.values[critique_index]][critiques.index.get_loc(movieName)]
        if math.isnan(numerator_e):
            break
        d_e = sim_distanceEuclidienne(critiques[nouveauCritique], critiques[critiques.columns.values[critique_index]])
        total_a_e += numerator_e / (1 + d_e)
        s_a_e += 1 / (1 + d_e)
        global_score_a_e = total_a_e / s_a_e
        
#     print("total_a_m = {}".format(total_a_m))
#     print("s_a_m = {}".format(s_a_m))
#     print("global_score_a_m = {}".format(global_score_a_m))
#     print()
#     print("total_a_e = {}".format(total_a_e))
#     print("s_a_e = {}".format(s_a_e))
#     print("global_score_a_e = {}".format(global_score_a_e))
    return global_score_a_m, global_score_a_e

Build the function Bestrecommend suggesting to Anne a recommendation between the three movies “Snakes”, “Superman” and “Night” based on global score.

In [290]:
def Bestrecommend_manhattan(person, critiques, array):
    
    score_manhattan = {}
    for i in range(len(not_seen_by_Anne(person, critiques, array))):
        score_manhattan[not_seen_by_Anne(person, critiques, array)[i]] = weighted_score(person, not_seen_by_Anne(person, critiques, array)[i], critiques)[0]
        
    return max(score_manhattan, key=score_manhattan.get)

print("Best recommended movie for {} with Manhattan distance - {}".format('Anne', Bestrecommend_manhattan('Anne', critiques1, movies)))

Best recommended movie for Anne with Manhattan distance - Superman


In [291]:
def Bestrecommend_euclidienne(person, critiques, array):
    
    score_euclidienne = {}
    for i in range(len(not_seen_by_Anne(person, critiques, array))):
        score_euclidienne[not_seen_by_Anne(person, critiques, array)[i]] = weighted_score(person, not_seen_by_Anne(person, critiques, array)[i], critiques)[1]
    
    return max(score_euclidienne, key=score_euclidienne.get)

print("Best recommended movie for {} with Euclidienne distance - {}".format('Anne', Bestrecommend_euclidienne('Anne', critiques1, movies)))

Best recommended movie for Anne with Euclidienne distance - Superman


In [292]:
def weighted_score_with_exp(nouveauCritique, movieName, critiques):
    
    #print(critiques1.iloc[[critiques1.index.get_loc(movieName)]]) #getting row by given movie name
    
    total_a_m = 0
    s_a_m = 0
    global_score_a_m = 0
    numerator_m = 0
    d_m = 0
    
    #for i in critiques1.columns.values:
        #print(critiques1[i][critiques1.index.get_loc(movieName)]) #getting value of each critique to given movie name
    
    for critique_index in range(len(critiques.columns.values)):
        numerator_m = critiques[critiques.columns.values[critique_index]][critiques.index.get_loc(movieName)]
        if math.isnan(numerator_m):
            break
        d_m = sim_distanceManhattan(critiques[nouveauCritique],critiques[critiques.columns.values[critique_index]])
        total_a_m += numerator_m / (1 + d_m)
        s_a_m += math.exp( -d_m )
        global_score_a_m = total_a_m / s_a_m

    total_a_e = 0
    s_a_e = 0
    global_score_a_e = 0
    numerator_e = 0
    d_e = 0
    
    for critique_index in range(len(critiques.columns.values)):
        numerator_e = critiques[critiques.columns.values[critique_index]][critiques.index.get_loc(movieName)]
        if math.isnan(numerator_e):
            break
        d_e = sim_distanceEuclidienne(critiques[nouveauCritique],critiques[critiques.columns.values[critique_index]])
        total_a_e += numerator_e / (1 + d_e)
        s_a_e += math.exp( -d_m )
        global_score_a_e = total_a_e / s_a_e
        
    return global_score_a_m, global_score_a_e

In [293]:
def BestrecommendwithExp_manhattan(person, critiques, array):
    
    score_manhattan_with_exp = {}
    for i in range(len(not_seen_by_Anne(person, critiques, array))):
        score_manhattan_with_exp[not_seen_by_Anne(person, critiques, array)[i]] = weighted_score_with_exp(person, not_seen_by_Anne(person, critiques, array)[i], critiques)[0]
        
    return max(score_manhattan_with_exp, key=score_manhattan_with_exp.get)
    
print("Best recommended movie for {} with Manhattan distance - {}".format('Anne', BestrecommendwithExp_manhattan('Anne', critiques1, movies)))

Best recommended movie for Anne with Manhattan distance - Night


In [294]:
def BestrecommendwithExp_euclidienne(person, critiques, array):
    
    score_euclidienne_with_exp = {}
    for i in range(len(not_seen_by_Anne(person, critiques, array))):
        score_euclidienne_with_exp[not_seen_by_Anne(person, critiques, array)[i]] = weighted_score_with_exp(person, not_seen_by_Anne(person, critiques, array)[i], critiques)[1]
    
    return max(score_euclidienne_with_exp, key=score_euclidienne_with_exp.get)

print("Best recommended movie for {} with Euclidienne distance - {}".format('Anne', BestrecommendwithExp_euclidienne('Anne', critiques1, movies)))

Best recommended movie for Anne with Euclidienne distance - Night


Build a function calculating pearson correlation coefficient between two persons.

In [295]:
def pearson(person1, person2):
    
    sum_xy = 0
    sum_x = 0
    sum_y = 0
    sum_x2 = 0
    sum_y2 = 0
    n = 0
    for key in person1:
        if key in person2:
            n += 1
            x = person1[key]
            y = person2[key]
            if math.isnan(x) or math.isnan(y):
                x = 0
                y = 0
            sum_xy += x*y
            sum_x += x
            sum_y += y
            sum_x2 += x**2
            sum_y2 += y**2
        denominator = sqrt(sum_x2 - (sum_x ** 2) / n) * sqrt(sum_y2 - (sum_y ** 2) / n)
    if denominator == 0:
        return 0
    else:
        return (sum_xy - (sum_x * sum_y) /n ) / denominator

Build a function suggesting to Anne, by using the Pearson correlation coefficient.

In [296]:
def PearsonRecommend(person, critiques, critiques1, array):
    
    pearson_recommend_dict = {}
    pearson_recommend_dict_best_movie = {}
    
    for i in critiques1.columns.values:
        if i != person:
            pearson_recommend_dict[i] = pearson(critiques[person], critiques[i])
        
#     print("Nearest neighbor by Pearson correlation coefficient - {}".format(max(pearson_recommend_dict, key=pearson_recommend_dict.get)))
#     print()

    pearson_recommend_dict_best = critiques[max(pearson_recommend_dict, key=pearson_recommend_dict.get)]
    
    for i in not_seen_by_Anne(person, critiques1, array):
        for j in pearson_recommend_dict_best:
            if i == j:
                pearson_recommend_dict_best_movie[i] = pearson_recommend_dict_best[i]
    
#     print(pearson_recommend_dict_best_movie)
#     print("Best recommended movie from {} - {}".format(max(pearson_recommend_dict, key=pearson_recommend_dict.get), max(pearson_recommend_dict_best_movie, key=pearson_recommend_dict_best_movie.get)))
    return max(pearson_recommend_dict_best_movie, key=pearson_recommend_dict_best_movie.get)

print("Best recommended movie for {} using Pearson correlation coefficient - {}".format('Anne', PearsonRecommend('Anne', critiques, critiques1, movies)))

Best recommended movie for Anne using Pearson correlation coefficient - Night


Build a function calculating cosine correlation coefficient between two persons.

In [297]:
def Cosine(person1, person2):
    
    sum_xy = 0
    sum_x = 0
    sum_y = 0
    
    for key in person1:
        if key in person2:
            x = person1[key]
            y = person2[key]
            
            if math.isnan(x) or math.isnan(y):
                x = 0
                y = 0
                
            sum_xy += x * y
            sum_x += x ** 2
            sum_y += y ** 2
            
        denominator = (sqrt(sum_x) * sqrt(sum_y))
    
    if denominator == 0:
        return 0
    else:
        return sum_xy / denominator

Build a function suggesting to Anne, by using the Cosine correlation coefficient.

In [298]:
def CosineRecommend(person, critiques, critiques1, array):
    
    cosine_recommend_dict = {}
    cosine_recommend_dict_best_movie = {}

    for i in critiques1.columns.values:
        if i != person:
            cosine_recommend_dict[i] = Cosine(critiques[person], critiques[i])
        
#     print("Nearest neighbor by Cosine correlation coefficient - {}".format(max(cosine_recommend_dict, key=cosine_recommend_dict.get)))
#     print()

    cosine_recommend_dict_best = critiques[max(cosine_recommend_dict, key=cosine_recommend_dict.get)]
    
    for i in not_seen_by_Anne(person, critiques1, array):
        for j in cosine_recommend_dict_best:
            if i == j:
                cosine_recommend_dict_best_movie[i] = cosine_recommend_dict_best[i]
    
#     print(cosine_recommend_dict_best_movie)
#     print("Best recommended movie from {} - {}".format(max(cosine_recommend_dict, key=cosine_recommend_dict.get), max(cosine_recommend_dict_best_movie, key=cosine_recommend_dict_best_movie.get)))
    return max(cosine_recommend_dict_best_movie, key=cosine_recommend_dict_best_movie.get)

print("Best recommended movie for {} using Cosine correlation coefficient - {}".format('Anne', CosineRecommend('Anne', critiques, critiques1, movies)))

Best recommended movie for Anne using Cosine correlation coefficient - Night


# 2. Music critics

Build a dictionary critiques containing the music critics and their ratings.

In [305]:
music_critiques = {}

music_critiques['Angelica'] = {'Blues Traveler': 3.5, 'Broken Bells': 2.0, 'Deadmau5': np.NaN, 'Norah Jones': 4.5, 'Phoenix': 5.0, 'Slightly Stupid': 1.5, 'The Strokes': 2.5, 'Vampire Weekend': 2.0}
music_critiques['Bill'] = {'Blues Traveler': 2.0, 'Broken Bells': 3.5, 'Deadmau5': 4.0, 'Norah Jones': np.NaN, 'Phoenix': 2.0, 'Slightly Stupid': 3.5, 'The Strokes': np.NaN, 'Vampire Weekend': 3.0}
music_critiques['Chan'] = {'Blues Traveler': 5.0, 'Broken Bells': 1.0, 'Deadmau5': 1.0, 'Norah Jones': 3.0, 'Phoenix': 5.0, 'Slightly Stupid': 1.0, 'The Strokes': np.NaN, 'Vampire Weekend': np.NaN}
music_critiques['Dan'] = {'Blues Traveler': 3.0, 'Broken Bells': 4.0, 'Deadmau5': 4.5, 'Norah Jones': np.NaN, 'Phoenix': 3.0, 'Slightly Stupid': 4.5, 'The Strokes': 4.0, 'Vampire Weekend': 2.0}
music_critiques['Hailey'] = {'Blues Traveler': np.NaN, 'Broken Bells': 4.0, 'Deadmau5': 1.0, 'Norah Jones': 4.0, 'Phoenix': np.NaN, 'Slightly Stupid': np.NaN, 'The Strokes': 4.0, 'Vampire Weekend': 1.0}
music_critiques['Jordyn'] = {'Blues Traveler': np.NaN, 'Broken Bells': 4.5, 'Deadmau5': 4.0, 'Norah Jones': 5.0, 'Phoenix': 5.0, 'Slightly Stupid': 4.5, 'The Strokes': 4.0, 'Vampire Weekend': 4.0}
music_critiques['Sam'] = {'Blues Traveler': 5.2, 'Broken Bells': 2.0, 'Deadmau5': np.NaN, 'Norah Jones': 3.0, 'Phoenix': 5.0, 'Slightly Stupid': 4.0, 'The Strokes': 5.0, 'Vampire Weekend': np.NaN}
music_critiques['Veronica'] = {'Blues Traveler': 3.0, 'Broken Bells': np.NaN, 'Deadmau5': np.NaN, 'Norah Jones': 5.0, 'Phoenix': 4.0, 'Slightly Stupid': 2.5, 'The Strokes': 3.0, 'Vampire Weekend': np.NaN}

music_critiques1 = pd.DataFrame(music_critiques)
songs = (music_critiques1.index)

music_critiques1

,Angelica,Bill,Chan,Dan,Hailey,Jordyn,Sam,Veronica
Blues Traveler,3.5,2.0,5.0,3.0,NaN,NaN,5.2,3.0
Broken Bells,2.0,3.5,1.0,4.0,4.0,4.5,2.0,NaN
Deadmau5,NaN,4.0,1.0,4.5,1.0,4.0,NaN,NaN
Norah Jones,4.5,NaN,3.0,NaN,4.0,5.0,3.0,5.0
Phoenix,5.0,2.0,5.0,3.0,NaN,5.0,5.0,4.0
Slightly Stupid,1.5,3.5,1.0,4.5,NaN,4.5,4.0,2.5
The Strokes,2.5,NaN,NaN,4.0,4.0,4.0,5.0,3.0
Vampire Weekend,2.0,3.0,NaN,2.0,1.0,4.0,NaN,NaN


# Which songs you can recommend to Veronica ?

To compute a simple similarity score, we use the Manhattan distance.

In [306]:
print("Similarity score by Manhattan distance")
print()

for i in music_critiques1.columns.values:
    for j in music_critiques1.columns.values:
        if i != j and i == 'Veronica':
            print("The similarity score between {} and {} = {}".format(i, j, sim_distanceManhattan(music_critiques1[i], music_critiques1[j])))

Similarity score by Manhattan distance

The similarity score between Veronica and Angelica = 3.5
The similarity score between Veronica and Bill = 4.0
The similarity score between Veronica and Chan = 6.5
The similarity score between Veronica and Dan = 4.0
The similarity score between Veronica and Hailey = 2.0
The similarity score between Veronica and Jordyn = 4.0
The similarity score between Veronica and Sam = 8.7


To compute a simple similarity score, we use an Euclidean distance.

In [307]:
print("Similarity score by Euclidean distance")
print()

for i in music_critiques1.columns.values:
    for j in music_critiques1.columns.values:
        if i != j and i == 'Veronica':
            print("The similarity score between {} and {} = {}".format(i, j, sim_distanceEuclidienne(music_critiques1[i], music_critiques1[j])))

Similarity score by Euclidean distance

The similarity score between Veronica and Angelica = 1.6583123951777
The similarity score between Veronica and Bill = 2.449489742783178
The similarity score between Veronica and Chan = 3.3541019662496847
The similarity score between Veronica and Dan = 2.449489742783178
The similarity score between Veronica and Hailey = 1.4142135623730951
The similarity score between Veronica and Jordyn = 2.449489742783178
The similarity score between Veronica and Sam = 4.011234224026316


Returning a sorted list of critics close to Veronica.

In [308]:
computeNearestNeighbor('Veronica', music_critiques1)

[(2.0, 'Hailey'),
 (3.5, 'Angelica'),
 (4.0, 'Bill'),
 (4.0, 'Dan'),
 (4.0, 'Jordyn'),
 (6.5, 'Chan'),
 (8.7, 'Sam')]

Returning one nearest critic to Veronica.

In [309]:
recommendNearestNeighbor('Veronica', music_critiques1, songs)

Nearest neighbor is Hailey.


[('Broken Bells', 4.0), ('Deadmau5', 1.0), ('Vampire Weekend', 1.0)]

Returning a list of music that Veronica didn't listen.

In [310]:
not_seen_by_Anne('Veronica', music_critiques1, songs)

['Broken Bells', 'Deadmau5', 'Vampire Weekend']

Suggesting to Veronica a recommendation between the musics based on global score.

In [311]:
print("Best recommended movie for {} with Manhattan distance - {}".format('Veronica', Bestrecommend_manhattan('Veronica', music_critiques1, songs)))
print("Best recommended movie for {} with Euclidienne distance - {}".format('Veronica', Bestrecommend_euclidienne('Veronica', music_critiques1, songs)))

Best recommended movie for Veronica with Manhattan distance - Broken Bells
Best recommended movie for Veronica with Euclidienne distance - Broken Bells


In [312]:
print("Best recommended movie for {} with Manhattan distance - {}".format('Veronica', BestrecommendwithExp_manhattan('Veronica', music_critiques1, songs)))
print("Best recommended movie for {} with Euclidienne distance - {}".format('Veronica', BestrecommendwithExp_euclidienne('Veronica', music_critiques1, songs)))

Best recommended movie for Veronica with Manhattan distance - Vampire Weekend
Best recommended movie for Veronica with Euclidienne distance - Broken Bells


Returning a suggestion to Veronica, by using the Pearson correlation coefficient.

In [313]:
print("Best recommended movie for {} using Pearson correlation coefficient - {}".format('Veronica', PearsonRecommend('Veronica', music_critiques, music_critiques1, songs)))

Best recommended movie for Veronica using Pearson correlation coefficient - Broken Bells


Returning a suggestion to Veronica, by using the Cosine correlation coefficient.

In [314]:
print("Best recommended movie for {} using Cosine correlation coefficient - {}".format('Veronica', CosineRecommend('Veronica', music_critiques, music_critiques1, songs)))

Best recommended movie for Veronica using Cosine correlation coefficient - Broken Bells


# Which songs you can recommend to Hailey ?

To compute a simple similarity score, we use the Manhattan distance.

In [320]:
print("Similarity score by Manhattan distance")
print()

for i in music_critiques1.columns.values:
    for j in music_critiques1.columns.values:
        if i != j and i == 'Hailey':
            print("The similarity score between {} and {} = {}".format(i, j, sim_distanceManhattan(music_critiques1[i], music_critiques1[j])))

Similarity score by Manhattan distance

The similarity score between Hailey and Angelica = 5.0
The similarity score between Hailey and Bill = 5.5
The similarity score between Hailey and Chan = 4.0
The similarity score between Hailey and Dan = 4.5
The similarity score between Hailey and Jordyn = 7.5
The similarity score between Hailey and Sam = 4.0
The similarity score between Hailey and Veronica = 2.0


To compute a simple similarity score, we use an Euclidean distance.

In [321]:
print("Similarity score by Euclidean distance")
print()

for i in music_critiques1.columns.values:
    for j in music_critiques1.columns.values:
        if i != j and i == 'Hailey':
            print("The similarity score between {} and {} = {}".format(i, j, sim_distanceEuclidienne(music_critiques1[i], music_critiques1[j])))

Similarity score by Euclidean distance

The similarity score between Hailey and Angelica = 2.7386127875258306
The similarity score between Hailey and Bill = 3.640054944640259
The similarity score between Hailey and Chan = 3.1622776601683795
The similarity score between Hailey and Dan = 3.640054944640259
The similarity score between Hailey and Jordyn = 4.387482193696061
The similarity score between Hailey and Sam = 2.449489742783178
The similarity score between Hailey and Veronica = 1.4142135623730951


Returning a sorted list of critics close to Hailey.

In [322]:
computeNearestNeighbor('Hailey', music_critiques1)

[(2.0, 'Veronica'),
 (4.0, 'Chan'),
 (4.0, 'Sam'),
 (4.5, 'Dan'),
 (5.0, 'Angelica'),
 (5.5, 'Bill'),
 (7.5, 'Jordyn')]

Returning one nearest critic to Hailey.

In [323]:
recommendNearestNeighbor('Hailey', music_critiques1, songs)

Nearest neighbor is Veronica.


[('Blues Traveler', 3.0), ('Phoenix', 4.0), ('Slightly Stupid', 2.5)]

Returning a list of music that Hailey didn't listen.

In [324]:
not_seen_by_Anne('Hailey', music_critiques1, songs)

['Blues Traveler', 'Phoenix', 'Slightly Stupid']

Suggesting to Hailey a recommendation between the musics based on global score.

In [325]:
print("Best recommended movie for {} with Manhattan distance - {}".format('Hailey', Bestrecommend_manhattan('Hailey', music_critiques1, songs)))
print("Best recommended movie for {} with Euclidienne distance - {}".format('Hailey', Bestrecommend_euclidienne('Hailey', music_critiques1, songs)))

Best recommended movie for Hailey with Manhattan distance - Phoenix
Best recommended movie for Hailey with Euclidienne distance - Phoenix


In [326]:
print("Best recommended movie for {} with Manhattan distance - {}".format('Hailey', BestrecommendwithExp_manhattan('Hailey', music_critiques1, songs)))
print("Best recommended movie for {} with Euclidienne distance - {}".format('Hailey', BestrecommendwithExp_euclidienne('Hailey', music_critiques1, songs)))

Best recommended movie for Hailey with Manhattan distance - Phoenix
Best recommended movie for Hailey with Euclidienne distance - Phoenix


Returning a suggestion to Hailey, by using the Pearson correlation coefficient.

In [327]:
print("Best recommended movie for {} using Pearson correlation coefficient - {}".format('Hailey', PearsonRecommend('Hailey', music_critiques, music_critiques1, songs)))

Best recommended movie for Hailey using Pearson correlation coefficient - Phoenix


Returning a suggestion to Hailey, by using the Cosine correlation coefficient.

In [328]:
print("Best recommended movie for {} using Cosine correlation coefficient - {}".format('Hailey', CosineRecommend('Hailey', music_critiques, music_critiques1, songs)))

Best recommended movie for Hailey using Cosine correlation coefficient - Phoenix


# 3. Elaborate movies recommendations

Build a function to get a value with 0.5 step.

In [329]:
def round_custom(num, step):
    return round(num / step) * step

In [470]:
def GenerateData(n, m):
    
    #create a condition, so that the function only works when the parameters are set correctly
    if n >= 10 and m >= 10:
        
        columns = [''] * n
        rows = [''] * m
        count_nan = 0
        
        for i in range(n): columns[i] = "c" + str(int(i))
        for j in range(m): rows[j] = "m" + str(int(j))
            
        rng = np.random.default_rng()
        gendata = pd.DataFrame(rng.uniform(0, 5, size = (m, n)), rows, columns)
        
        for i in rows:
            for j in columns:
                gendata.loc[i][j] = round_custom(gendata.loc[i][j], 0.5)
                if gendata.loc[i][j] == 0.0:
                    count_nan += 1

        for critique_index in range(len(gendata.columns.values)):
            for critique_row in range(len(gendata.index.values)):
                if count_nan / (n * m) <= 0.3 or count_nan / (n * m) >= 0.5:
                    new_n = 0
                    new_m = 0
                    new_n = random.randint(1, n-1)
                    new_m = random.randint(1, m-1)
                    if(gendata[gendata.columns.values[new_n]][gendata.index.values[new_m]] != 0.0):
                        gendata[gendata.columns.values[new_n]][gendata.index.values[new_m]] = 0.0
                        count_nan +=1

        gendata1 = gendata.to_dict()
        mov = (gendata.index)
        
        count = 0
        critic5more = {}
        person = ''
        
        for i in gendata1:
            for j in gendata1[i]:
                if gendata1[i][j] == 0.0:
                    count +=1
            if count >= m/2:
                critic5more = gendata1[i]
            count = 0
            
        for i in rows:
            for j in columns:
                if gendata.loc[i][j] == 0.0:
                    gendata.loc[i][j] = np.nan

        for i in gendata.columns.values:
            if gendata1[i] == critic5more:
                person = i

        a = Bestrecommend_manhattan(person, gendata, mov)
        b = BestrecommendwithExp_manhattan(person, gendata, mov)
        c = BestrecommendwithExp_euclidienne(person, gendata, mov)
        d = PearsonRecommend(person, gendata1, gendata, mov)
        e = CosineRecommend(person, gendata1, gendata, mov)
        
        if a != b and a != c and a != d and a != e and b != c and b != d and b != e and c != e:
            print("Best recommended movie for {} with Manhattan distance - {}".format(person, a))
            print("Best recommended with Exp movie for {} with Manhattan distance - {}".format(person, b))
            print("Best recommended with Exp movie for {} with Euclidienne distance - {}".format(person, c))
            print("Best recommended movie for {} using Pearson correlation coefficient - {}".format(person, d))
            print("Best recommended movie for {} using Cosine correlation coefficient - {}".format(person, e))
            print(gendata)
            
        else: 
            GenerateData(n, m)

    else:
        return ("Please try again! n and m should be bigger than 10" )
    
n = input("Type a number of critics (n): ")
m = input("Type a number of movies (m): ")

GenerateData(int(n), int(m))

Type a number of critics (n): 10
Type a number of movies (m): 10
Best recommended movie for c6 with Manhattan distance - m4
Best recommended with Exp movie for c6 with Manhattan distance - m1
Best recommended with Exp movie for c6 with Euclidienne distance - m3
Best recommended movie for c6 using Pearson correlation coefficient - m5
Best recommended movie for c6 using Cosine correlation coefficient - m5
     c0   c1   c2   c3   c4   c5   c6   c7   c8   c9
m0  NaN  1.5  0.5  3.0  4.0  2.0  4.5  1.0  3.5  2.5
m1  3.0  NaN  4.0  4.5  1.5  NaN  NaN  4.0  1.0  2.0
m2  1.5  NaN  0.5  NaN  2.5  4.0  4.5  2.0  2.5  NaN
m3  1.0  0.5  0.5  NaN  4.5  0.5  NaN  4.5  2.0  3.0
m4  3.0  4.0  3.0  2.5  NaN  1.0  NaN  4.5  NaN  5.0
m5  1.0  NaN  1.5  3.0  2.5  4.0  NaN  2.5  3.0  NaN
m6  3.5  NaN  NaN  1.5  NaN  2.5  4.0  NaN  NaN  4.0
m7  1.5  NaN  3.0  4.0  NaN  1.5  2.5  1.0  1.5  NaN
m8  1.0  NaN  2.5  NaN  NaN  4.5  2.5  NaN  5.0  0.5
m9  2.0  NaN  3.5  NaN  3.5  4.0  NaN  3.0  NaN  1.5
